# <font color='red'><b>Implementation of Radom Forests</b> </font>

<font color='blue'> <b>Importing packages</b> </font>

In [ ]:
import numpy as np # importing numpy for numerical computation
from sklearn.datasets import load_boston # here we are using sklearn's boston dataset
from sklearn.metrics import mean_squared_error # importing mean_squared_error metric
import random
from statistics import mean
from statistics import stdev

In [ ]:
boston = load_boston()                                                          
x=boston.data #independent variables
y=boston.target #target variable

In [ ]:
x.shape

(506, 13)

In [ ]:
x[0][:10]

array([6.320e-03, 1.800e+01, 2.310e+00, 0.000e+00, 5.380e-01, 6.575e+00,
       6.520e+01, 4.090e+00, 1.000e+00, 2.960e+02])

In [ ]:
x[:5,:3]

array([[6.320e-03, 1.800e+01, 2.310e+00],
       [2.731e-02, 0.000e+00, 7.070e+00],
       [2.729e-02, 0.000e+00, 7.070e+00],
       [3.237e-02, 0.000e+00, 2.180e+00],
       [6.905e-02, 0.000e+00, 2.180e+00]])

In [ ]:
l= [1,3,5,6,7,10]

In [ ]:
row= list(range(len(x)))
print(row[:10])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [ ]:
col= list(range(len(x[0])))
print(col)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


<font color='blue'> <b>Generating samples</b></font>

In [ ]:
import random
from random import sample
import numpy as np
from tqdm import tqdm                                                              
from sklearn.tree import DecisionTreeRegressor

In [ ]:
def generating_samples(input_data, target_data):

    row1= list(range(len(input_data)))
    sr_x= sample(row1,303)

    col= list(range(3,13))

    r_num= random.randint(3,10)

    c_x= sample(col,r_num)

    x_data= input_data[sr_x]
    sx_data= x_data[:,c_x]

    sy_data= target_data[sr_x]

    row2= list(range(len(sx_data)))

    rr_x= sample(row2,203)

    rsx_data= sx_data[rr_x]
    ry_data= sy_data[rr_x]

    final_sample_data= np.vstack((sx_data,rsx_data))
    final_target_data= np.vstack((sy_data.reshape(-1,1),ry_data.reshape(-1,1)))

    return final_sample_data,final_target_data,sr_x,c_x

<font color='blue'> <b>Creating 30 samples </b>

In [ ]:
# Use generating_samples function to create 30 samples 
# store these created samples in a list
def selections(x,y):
    list_input_data =[]
    list_output_data =[]
    list_selected_row= []
    list_selected_columns=[]

    for i in tqdm(range(0,30)):
        a,b,c,d= generating_samples(x,y)
        list_input_data.append(a)
        list_output_data.append(b)
        list_selected_row.append(c)
        list_selected_columns.append(d)
    
    return list_input_data,list_output_data,list_selected_row,list_selected_columns

In [ ]:
list_input_data_f,list_output_data_f,list_selected_row_f,list_selected_columns_f= selections(x,y)

100%|██████████| 30/30 [00:00<00:00, 1409.66it/s]


<font color='blue'><b> Code for building regression trees</b></font>

In [ ]:
def build_trees(list_input_data,list_output_data,list_selected_row,list_selected_columns):
    list_of_all_models= []

    for i in tqdm(range(len(list_input_data))):
        reg= DecisionTreeRegressor()
        model= reg.fit(list_input_data[i],list_output_data[i])
        list_of_all_models.append(model)
    return list_of_all_models


In [ ]:
list_of_all_models_f= build_trees(list_input_data_f,list_output_data_f,list_selected_row_f,list_selected_columns_f)

100%|██████████| 30/30 [00:00<00:00, 450.97it/s]


<font color='blue'><b> Code for calculating MSE on the TRAIN Data</b></font>

In [ ]:
def mse_calc(list_of_all_models,x,y,list_selected_columns):
    array_of_y=[]
    for i in tqdm(range(len(list_of_all_models))):
        y=[]
    
        for j in range(len(x)):
            y.append(list_of_all_models[i].predict(x[j][list_selected_columns[i]].reshape(1,-1)))

        array_of_y.append(y)
    
    y_pred=[]
    for k in tqdm(range(len(x))):
        g=0
        for l in range(len(list_of_all_models)):
            g= g+array_of_y[l][k][0]
        y_pred.append(g/len(list_of_all_models))
    
    mse=0

    for i in tqdm(range(len(y))):
        mse=mse+((y[0]-y_pred[0])**2)
    mse= mse/len(y)
    
    return mse,y_pred

In [ ]:
MSE,hp_pred= mse_calc(list_of_all_models_f,x,y,list_selected_columns_f)

100%|██████████| 506/506 [00:00<00:00, 148237.61it/s]


In [ ]:
print(MSE[0])

0.8673817777777789


In [ ]:
type(hp_pred)

list

In [ ]:
print("The House Price predicted on Train Data is:",mean(hp_pred))

The House Price predicted on Train Data is: 22.551062481458583


<font color='blue'><b> Calculating OOB score </b></font>

In [ ]:
x_r= list(range(506))

In [ ]:
def oob_score(list_of_all_models,list_selected_row,list_selected_columns,x,y):
    y_pred_oob=[]
    oob_output_data=[]

    for i in tqdm(range(len(list_of_all_models))):
        oobs= list(set(x_r)-set(list_selected_row[i]))
        y_o=[]
        outputs=[]
    
    for j in range(len(oobs)):
        y_o.append(list_of_all_models[i].predict(x[j][list_selected_columns[i]].reshape(1,-1)))
    y_pred_oob.append(y_o)

    for k in range(len(oobs)):
        outputs.append(y[oobs[k]])
    oob_output_data.append(outputs)

    oob_score=0
    
    for m in tqdm(range(len(oob_output_data))):
        o_s=0
        for n in range(len(oob_output_data[m])):
            o_s= o_s+((y_pred_oob[m][n]-oob_output_data[m][n])**2)[0]
            o_s= o_s/len(oob_output_data[m])
        oob_score= oob_score+o_s 
    
    oob_score= oob_score/len(oob_output_data)

    return oob_score,y_pred_oob


In [ ]:
OOBScore, hp_oob_pred= oob_score(list_of_all_models_f,list_selected_row_f,list_selected_columns_f,x,y)

100%|██████████| 1/1 [00:00<00:00, 376.07it/s]


In [ ]:
print(OOBScore)

1.6679491928028818


In [1]:
from prettytable import PrettyTable
P= PrettyTable()

P.field_names= ["DataSet","Predicted House Price","MSE"]
P.add_row(["Train Data",22.55,0.86])
P.add_row(["OOB Data",24,1.66])

print(P)

+------------+-----------------------+------+
|  DataSet   | Predicted House Price | MSE  |
+------------+-----------------------+------+
| Train Data |         22.55         | 0.86 |
|  OOB Data  |           24          | 1.66 |
+------------+-----------------------+------+


<font color='blue'><b>Observations:</b></font>
 
We see that the mean house price predicted on the in-bag part of the data is 22.55 with a mean squared error of 0.86 which means that the house prices would range from 21.69 to 23.41. Where the house price predicted by the Out of Bag (OOB) data is 24 with a MSE of 1.66, which means that the house price could vary between 22.34 and 25.66. We see a major overlap between both the inbag data and the out of bag data predicted house prices. 

Hence we can say that the model is very well generalized and performs equally well on both the training data and the new data(Out of Bag data) as the MSE's of inbag and out of bag is 0.86 and 1.66 which are very close to each other.